# Routines for building open cluster graphics based on Gaia DR2 data

## Author: Bruno Sampaio Alessi
## Since: Aug 09 2020

## 1 - Libraries

In [1]:
#%matplotlib notebook

import gc
import sys
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import pi
from astropy import units
from astropy.table import Table
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord


## 2 - Input / Output

In [2]:
# folders, files, etc
# you must have these directories inside the directory which this notebook is located:
# /input/data and /input/graphics
root_dir   = os.path.join('.','io_files')
input_file = os.path.join(root_dir, 'clusters.csv')
data_dir   = os.path.join(root_dir, 'data')
graph_dir  = os.path.join(root_dir, 'graphics')
estim_file = os.path.join(root_dir, 'derived_params.tsv')

# input file headers
headers = ('name', 'const', 'ra_deg', 'de_deg', 'rad_deg', 'g_mag_lim', 'large_pm_flag', 'large_plx_flag')
convs   = {k:str.strip for k in headers}

# get filename based on the cluster name in the input file
def get_fname(_dir, name, ext):
    fname = name.replace(' ','_') + '.' + ext
    return os.path.join(_dir, fname)

# reads the input file and loads its contents in a DataFrame
def get_input_dataframe():
    df = pd.read_csv(input_file, converters = convs)
    df = df.set_index('name')
    df["ra_deg"] = pd.to_numeric(df["ra_deg"])
    df["de_deg"] = pd.to_numeric(df["de_deg"])
    df["rad_deg"] = pd.to_numeric(df["rad_deg"])
    return df

# uncomment to check DF contents
df = get_input_dataframe()
#print(df.dtypes)
df.head()


,const,ra_deg,de_deg,rad_deg,g_mag_lim,large_pm_flag,large_plx_flag
name,,,,,,,
Loden 143,Car,157.183333,-58.821667,0.045833,<15,,
Ruprecht 159,Car,140.035417,-60.403889,0.012500,<18,,
Ruprecht 163,Car,166.196667,-67.949167,0.018333,<17,,
Ruprecht 89,Car,157.110417,-58.179167,0.033333,<15,,
Ruprecht 90,Car,157.750000,-58.456944,0.100000,<14,,


## 3 - Query Gaia DR2 Catalog

In [3]:
# Gaia DR2 catalog on VizieR
# see https://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=I/345/gaia2
cat = 'I/345/gaia2'
cols = ['_r', 'DR2Name', 'RAdeg', 'DEdeg', 'Plx', 'e_Plx', 'pmRA', 'pmDE', 'Gmag', 'BP-G', 'RV']

#Defines the query contraints
def get_constraints(name, input_df):
    filters = dict()
    filters['Plx']   = '-0.5..5'
    filters['pmRA']  = '-15..15'
    filters['pmDE']  = filters['pmRA']
    filters['Gmag']  = input_df.loc[name,'g_mag_lim']
    filters['e_Plx'] = '<0.5'
    return filters

#Returns the coordinates and radius of the area to be queried
def get_locus(name, input_df):
    ra_deg = input_df.loc[name,'ra_deg']
    de_deg = input_df.loc[name,'de_deg']
    radius = input_df.loc[name,'rad_deg'] * units.degree
    return SkyCoord(ra = ra_deg*units.degree, dec = de_deg*units.degree, frame = 'icrs'), radius

#Query the GDR2 data for one cluster
def query(name, input_df):
    fname = get_fname(data_dir, name, 'tsv')
    #checks if the GDR2 data was previously saved locally
    #ATTENTION! if you want to re-generate output files (graphics, etc) with other
    #set of parameters (e.g. constraints), you need to delete the previously saved files
    if os.path.exists(fname):
        print("Reading existing data for %s ..." % name)
        return pd.read_csv(fname)
    print("Querying VizieR for %s ..." % name)
    filters  = get_constraints(name, input_df)
    coo, rad = get_locus(name, input_df)
    viz = Vizier(columns = cols, column_filters = filters, row_limit = 9999, timeout = 300)
    viz_res = viz.query_region(coo, radius = rad, catalog = cat)
    if viz_res == None or len(viz_res) == 0:
        return pd.DataFrame() #Empty Dataframe
    return viz_res[0].to_pandas()

# TODO:
# 1 - create a function to query only one cluster given it's radius and coordinates
# 2 - query a larger area to generate a proper RDP (see comments in cell 8)

#Save the queried data
def save(name, output_df, max_stars = 200):
    fname = get_fname(data_dir, name, 'tsv')
    output_df = output_df.set_index('DR2Name')
    output_df = output_df.sort_values(by='Gmag')
    output_df = output_df[:max_stars]
    output_df = output_df.sort_values(by='_r')
    headers = list(output_df.columns.values)
    output_df = output_df[headers[1:] + headers[:1]] #put RV in the last position
    if not os.path.exists(fname):
        print("Writing file %s..." % fname)
        output_df.to_csv(fname, float_format='% .6f')
    return output_df


## 4 - Auxiliary routines for tidying graphics

In [4]:
# Maybe it's better to use %-iles to limit and tidy the graphics
# instead of setting those clumsy limits - use dataframe.describe?
def get_lim_pm(max_pm):
    return 15 if max_pm < 15 else \
           20 if max_pm < 20 else \
           25 if max_pm < 25 else \
           30 if max_pm < 30 else \
           35
            
# Maybe it's better to use %-iles to limit and tidy the graphics
# instead of setting those clumsy limits - use dataframe.describe?
def get_lim_plx(max_plx):
    return 4 if max_plx < 4 else \
           5 if max_plx < 5 else \
           6

def filter_pms(pmra, pmde):
    return not(-10 < pmra < 10 and -10 < pmde < 10)

def filter_plxs(plx):
    return plx > 2

def pm_limits(pmras, pmdes):
    max_pm = np.max([np.abs(np.min(pmras)), np.abs(np.max(pmras))])
    lim_pm = get_lim_pm(max_pm)
    min_pmRA = -1 * lim_pm
    max_pmRA = lim_pm
    max_pm = np.max([np.abs(np.min(pmdes)), np.abs(np.max(pmdes))])
    lim_pm = get_lim_pm(max_pm)
    min_pmDe = -1 * lim_pm
    max_pmDe = lim_pm
    return min_pmRA, max_pmRA, min_pmDe, max_pmDe
 
def plx_limits(plxs, min_plx = -0.5):
    max_plx = np.max(plxs)
    max_plx = get_lim_plx(max_plx)
    return min_plx, max_plx

def mag_limits(gmags, color_mag_incr = 0.5):
    min_gmag = np.min(gmags) - color_mag_incr
    max_gmag = np.max(gmags) + color_mag_incr
    return min_gmag, max_gmag


## 5 - VPD scatter plot

In [5]:
# Build a VPD (Vector Proper Motion Diagram) scatter plot
def plot_VPD(data, title = None):
    pmras = data['pmRA']
    pmdes = data['pmDE']
    min_pmRA, max_pmRA, min_pmDe, max_pmDe = pm_limits(pmras, pmdes)
    xs, ys = pmras, pmdes
    if title:
        plt.title(title)
    plt.xlabel('pmRA (mas/yr)')
    plt.ylabel('pmDe (mas/yr)')
    plt.xlim(min_pmRA, max_pmRA)
    plt.ylim(min_pmDe, max_pmDe)
    plt.grid(True, linestyle='dashed')
    plt.scatter(xs, ys, s=3, color='black')

    
# TODO (for this and all graphics below):
# create a function generate a unique graphic (instead of grouping it with the other five)
# for only one cluster passing the # cluster name, radius and coordinates


## 6 - Parallax x G mag scatter plot

In [6]:
# Build G mag X Parallax scatter plot
def plot_mag_par(data, title = None):
    plxs = data['Plx']
    gmags = data['Gmag']
    min_gmag, max_gmag = mag_limits(gmags)
    min_plx, max_plx = plx_limits(plxs)
    xs, ys = gmags, plxs
    if title:
        plt.title(title)
    plt.xlabel('G (mag)')
    plt.ylabel('Plx (mas)')
    plt.xlim(min_gmag, max_gmag)
    plt.ylim(min_plx, max_plx)
    plt.grid(True, linestyle='dashed')
    plt.scatter(xs, ys, s=3, color='black')


## 7 - CMD scatter plot

In [7]:
# Build CMD (Color-Magnitude Diagram) scatter plot
def plot_CMD(data, title = None):
    gmags = data['Gmag']
    colors = data['BP-G']
    min_gmag, max_gmag = mag_limits(gmags)
    min_color, max_color = mag_limits(colors)
    xs, ys = colors, gmags
    if title:
        plt.title(title)
    plt.xlabel('BP-G (mag)')
    plt.ylabel('G (mag)')
    plt.xlim(min_color, max_color)
    plt.ylim(max_gmag, min_gmag)
    plt.grid(True, linestyle='dashed')
    plt.scatter(xs, ys, s=3, color='black')


## 8 - RDP scatter plot

In [8]:
# TODO
# This section is unfinished - the generated graphics is fixed and the routines are preliminary
# It is necessary to query an area larger than the area defined in cell 3
# and draw all graphics but the RDP using the radius and the RDP using the larger
# area (an ideally a neighbouring field to trace a line representing the average
# background density)


## 9 - Parallax Histogram

In [9]:
# Build Histogram for parallaxes
def plot_plx_hist(data, is_lg_plx, title = None, step = 0.25):
    plxs = data['Plx']
    xs = data[data.apply(lambda x: filter_plxs(x['Plx']),axis=1)]['Plx'] if is_lg_plx else plxs
    min_plx, max_plx = plx_limits(xs)
    bins = np.arange(min_plx, max_plx, step)
    if title:
        plt.title(title)
    plt.ylabel('N')
    plt.xlabel('Plx (mas)')
    plt.hist(xs, bins, histtype='bar', ec='black')
    #counts, bins, _ = plt.hist(xs, bins, histtype='bar', ec='black')
    #idx = np.unravel_index(np.argmax(counts), counts.shape)[0]
    #return (bins[idx] + bins[idx+1]) / 2


## 10 - Proper Motion 2D Histogram (returning PMs for the bin with highest count)

In [10]:
# Calculates the median parallax of the stars within <limit> of the estimated pms
def calculate_estimates(data, est_pmra, est_pmde, limit = 3):
    filter_fn = lambda x: filter_pms(x['pmRA'],x['pmDE'], est_pmra, est_pmde, limit )
    data = data[data.apply(filter_fn, axis = 1)]
    plxs = data['Plx']
    pmra = data['pmRA']
    pmde = data['pmDE']
    return np.median(pmra), np.median(pmde), np.median(plxs)

def filter_pms(pmra, pmde, est_pmra, est_pmde, limit):
    return (abs(est_pmra - pmra) < limit) and (abs(est_pmde - pmde) < limit)

# Build 2D histogram for proper motions and returns average PMs for the bin with the highest count
def plot_pm_hist(data, is_lg_pm, title = None, num_bins = 20):
    data = data[data.apply(lambda x: filter_pms(x['pmRA'],x['pmDE']), axis = 1)] if is_lg_pm else data
    xs = data['pmRA']
    ys = data['pmDE']
    min_pmRA, max_pmRA, min_pmDe, max_pmDe = pm_limits(xs, ys)
    counts, xedges, yedges = np.histogram2d(xs, ys, bins = num_bins, range = [[min_pmRA, max_pmRA], [min_pmDe, max_pmDe]])
    x_ind, y_ind = np.unravel_index(np.argmax(counts), counts.shape)
    est_pmra = (xedges[x_ind] + xedges[x_ind+1]) / 2
    est_pmde = (yedges[y_ind] + yedges[y_ind+1]) / 2
    plt.imshow(counts.T, origin = 'lower')
    if title:
        plt.title(title)
    plt.xlabel('pmRA bin index')
    plt.ylabel('pmDE bin index')
    plt.plot(x_ind, y_ind, 'or')
    est_pmra, est_pmde, est_plx = calculate_estimates(data, est_pmra, est_pmde)
    return est_pmra, est_pmde, est_plx


## 11 - Chart scatter plot

In [11]:
# TODO
# Create cluster chart scatter plot (Ra x Dec, in degrees)

## 12 - Plot all graphics as subplots (and calculates Plx estimate based on PMs in step 10)

In [12]:
# Plot everything in a 3 x 3 grid and returns a crude estimate for the cluster proper motion
# based on the bin with highest count in the 2D histogram of proper motions
def plot(cl_name, input_df, data):
    fig = plt.figure(figsize=(15,10))
    
    plt.subplot(231)
    plot_VPD(data)
    
    plt.subplot(232)
    is_lg_pm = input_df.loc[cl_name,'large_pm_flag']
    est_pmra, est_pmde, est_plx = plot_pm_hist(data, is_lg_pm, cl_name)
    estimates = f'{cluster_name:<28}; {est_plx:5.2f}; {est_pmra:6.2f}; {est_pmde:6.2f}\n'
    
    plt.subplot(233)
    plot_CMD(data)
    
    plt.subplot(234)
    plot_mag_par(data)
    
    plt.subplot(235)
    is_lg_plx = input_df.loc[cl_name, 'large_plx_flag']
    title = f'estimates (plx, pmra, pmde) = ({est_plx:5.2f}, {est_pmra:6.2f}, {est_pmde:6.2f})'
    plot_plx_hist(data, is_lg_plx, title)
    
    #plt.subplot(236)
    #plot_RDP(data)
    
    plt.tight_layout()
    file_path_name = get_fname(graph_dir, cl_name, 'jpg')
    fig.savefig(file_path_name, dpi = 100)
    plt.close('all')
    return estimates


## 13 - Main routines

In [13]:
def process_cluster(cluster_name, input_df, min_stars = 5):
    estimate = None
    # Query Vizier or read from csv file if previously saved
    output_df = query(cluster_name, input_df)
    # If the cluster has less than this number, the program will not plot its graphics
    if len(output_df.index) < min_stars:
        print("No data found for %s or it contains less than %d stars" % (cluster_name, min_stars))
    else:
        # Tidy and save data in CSV file if not already saved
        data = save(cluster_name, output_df)
        # Plot graphics and generate estimates
        estimate = plot(cluster_name, input_df, data)
        del data
    del output_df
    return estimate

def save_estimates(estimates):
    if estimates:
        #Do not manipulate files directly - put this in a dataframe
        print('Writing output file %s with plx and pm estimates...' % estim_file)
        with open(estim_file, 'w', encoding='utf8') as f:
            f.write('Name;Plx_est;PMRA_est;PMDe_est\n')
            for estimate in estimates:
               f.write(estimate)

#FIXME:
# There is a memory leak that shows if you try to process ~200 clusters ...
# I deleted the data frame on each iteration, closed the graphics and
# called the garbage collector and the leak still persists

cl_estimates = []
print('Reading input parameters from %s ...' % input_file)
input_df = get_input_dataframe()
print('Generating graphics...')
for cluster_name in input_df.index:
    try:
        estimate = process_cluster(cluster_name, input_df)
        if estimate:
            cl_estimates.append(estimate)
        gc.collect()
    except:
        _type, value, trace = sys.exc_info()
        print('Error processing %s: type=%s, value=%s' % (cluster_name, _type, value))
        print(trace)
        break
save_estimates(cl_estimates)

print("That's all, folks!")


Reading input parameters from .\io_files\clusters.csv ...
Generating graphics...
Querying VizieR for Loden 143 ...
Writing file .\io_files\data\Loden_143.tsv...
Querying VizieR for Ruprecht 159 ...
Writing file .\io_files\data\Ruprecht_159.tsv...
Querying VizieR for Ruprecht 163 ...
Writing file .\io_files\data\Ruprecht_163.tsv...
Querying VizieR for Ruprecht 89 ...
Writing file .\io_files\data\Ruprecht_89.tsv...
Querying VizieR for Ruprecht 90 ...
Writing file .\io_files\data\Ruprecht_90.tsv...
Writing output file .\io_files\derived_params.tsv with plx and pm estimates...
That's all, folks!
